In [14]:
import requests, praw, re, time
from bs4 import BeautifulSoup 
import pandas as pd 
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from config import r_client_id,r_secret_key,r_dev,r_username,r_pw

In [15]:
#naturla language took kit
nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\citiz\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\citiz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
reddit = praw.Reddit(
    client_id=r_client_id,
    client_secret=r_secret_key,
    password=r_pw,
    user_agent=f'testscript by u/{r_dev}',
    username=r_dev,
)
#print(reddit.user.me())

In [3]:
#FUNCTION to convert time stamp into something readable
def time_convertion(time):
    #time=str(time)
    time = datetime.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')
    time = pd.to_datetime(time)
    return time
print(time_convertion(1.622660e+09))

2021-06-02 18:53:20


In [4]:
#FUNCTION to pars stock symbols out of comment body
def symbol_pars(s):
    match= re.findall('[A-Z\s]{3,4}', s)
    Match_split = ','.join(match)
    return Match_split

In [6]:
#loop through subreddit Wallstreet Bets' Daily Discussion Thred, append into a list of dictionaries 
for submission in reddit.subreddit("wallstreetbets").search('Daily Discussion Thread', limit=4, sort='new'):
    #print(submission.title)
    comment_lst = []
    submission.comment_sort = 'new'
    for comment in submission.comments:
        try:
            #print(comment.body)
            d = {'Comment_Body':comment.body,
                'Author':comment.author,
                'Score':comment.score,
                'Time': time_convertion(comment.created_utc),
                'Symbol': symbol_pars(comment.body)
                }
            comment_lst.append(d)
        except:
            pass
#turns our list of dictionaries into a dataframe. Sort values by time posted
df = pd.DataFrame(comment_lst)
df.sort_values('Time', ascending=False)

,Comment_Body,Author,Score,Time,Symbol
1,If ya'll hopped on CLNE yesterday I'll see you...,DonBaron,2,2021-06-04 13:05:36,"CLN, APE"
2,You all realize when you make these limit buys...,No-Percentage-3786,2,2021-06-03 19:43:50,
3,"Why is the boss of memestocks, GME, not doing ...",MetalliTooL,1,2021-06-03 19:00:46,GME
4,CAN BB JUST GO UP OR DOWN WITHOUT AMC,SeagullMan2,3,2021-06-03 18:56:20,"CAN ,BB J,UST ,GO U,P OR, DOW,N WI,THOU,T AM"
5,Qgen about to go,cacy1800,2,2021-06-03 18:29:42,
...,...,...,...,...,...
119,RKT TO THE MOON! Let’s not forget about AMC 📈🍿,Jstayflexinn__,-10,2021-06-02 21:01:23,"RKT ,TO T,HE M,OON, AMC"
120,RKT TO THE MOON! Let’s not forget about AMC 📈🍿,Jstayflexinn__,-11,2021-06-02 21:01:12,"RKT ,TO T,HE M,OON, AMC"
121,Tilray beginning the move🚀,ForsakenSituation713,15,2021-06-02 21:00:57,
122,RLT 🚀🚀🚀,l3all3r900,2,2021-06-02 21:00:17,RLT


In [13]:
#locate comments that contain BB 
test = df.loc[df['Comment_Body'].str.contains('BB')]
test.head()

,Comment_Body,Author,Score,Time,Symbol
4,CAN BB JUST GO UP OR DOWN WITHOUT AMC,SeagullMan2,3,2021-06-03 18:56:20,"CAN ,BB J,UST ,GO U,P OR, DOW,N WI,THOU,T AM"
9,Bought the BB dip. Suck it gaybears,NicestGuyOnHere,6,2021-06-03 13:53:15,BB
11,BB Apes: Buy OTM June 18 40 calls to keep thi...,154hpc,4,2021-06-03 13:38:24,"BB A, B, OTM, MM , EPI"
12,Everyone selling amc and buying BB 🔥,Such_Dragonfruit_298,4,2021-06-03 13:34:00,BB
16,Not pushing ( SNDL )just asking I’m holding on...,OlympusChimera,10,2021-06-03 10:35:51,"SND, GME, AMC, BB"


In [9]:
df.head()

,Comment_Body,Author,Score,Time,Symbol
0,What are y'alls thoughts on these generic movi...,Dan_inKuwait,1,2021-06-02 10:34:08,"\n\nP, I , \n\n, I ,\n\nE, \n\n, \n\n,\n\n\n"
1,If ya'll hopped on CLNE yesterday I'll see you...,DonBaron,2,2021-06-04 13:05:36,"CLN, APE"
2,You all realize when you make these limit buys...,No-Percentage-3786,2,2021-06-03 19:43:50,
3,"Why is the boss of memestocks, GME, not doing ...",MetalliTooL,1,2021-06-03 19:00:46,GME
4,CAN BB JUST GO UP OR DOWN WITHOUT AMC,SeagullMan2,3,2021-06-03 18:56:20,"CAN ,BB J,UST ,GO U,P OR, DOW,N WI,THOU,T AM"


In [8]:
#locate comments that include AMC
AMCLoc = df.loc[df['Comment_Body'].str.contains('AMC') ]
AMCLoc.head()

,Comment_Body,Author,Score,Time,Symbol
4,CAN BB JUST GO UP OR DOWN WITHOUT AMC,SeagullMan2,3,2021-06-03 18:56:20,"CAN ,BB J,UST ,GO U,P OR, DOW,N WI,THOU,T AM"
16,Not pushing ( SNDL )just asking I’m holding on...,OlympusChimera,10,2021-06-03 10:35:51,"SND, GME, AMC, BB"
21,Feel I've missed the boat with GME/AMC. What s...,SignificantGiraffe5,3,2021-06-03 08:44:23,"GME,AMC, I"
22,What do you reckon will happen with AMC today?...,x_amber,3,2021-06-03 08:33:51,AMC
33,We’re gonna have a lot of cannabis millionaire...,Stup1dStonks,1,2021-06-03 01:39:13,"\n\nT, \nAM, \nBB"


In [26]:
sia = SIA()
   
scores = sia.polarity_scores('I love cats ')
scores

{'neg': 0.0, 'neu': 0.192, 'pos': 0.808, 'compound': 0.6369}